## 复现metaml t2d分析过程

In [1]:
import warnings
warnings.filterwarnings("ignore")
from numpy.core.umath_tests import inner1d
from sklearn.svm import SVC
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import LassoCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
#from sklearn.cross_validation import StratifiedKFold
from sklearn import preprocessing as prep
from sklearn import metrics
from sklearn import decomposition
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import argparse as ap
from sklearn.utils.multiclass import type_of_target
matplotlib.use('Agg')

```
python classification.py -h
usage: classification.py [-h] [-z FEATURE_IDENTIFIER] [-d DEFINE] [-t TARGET]
                         [-u UNIQUE] [-b] [-r RUNS_N] [-p RUNS_CV_FOLDS] [-w]
                         [-l {rf,svm,lasso,enet}] [-i {lasso,enet}]
                         [-f CV_FOLDS] [-g CV_GRID] [-s CV_SCORING]
                         [-j FS_GRID] [-e FIGURE_EXTENSION]
                         [INPUT_FILE] [OUTPUT_FILE]

positional arguments:
  INPUT_FILE            The input dataset file [stdin if not present]
  OUTPUT_FILE           The output file [stdout if not present]

optional arguments:
  -h, --help            show this help message and exit
  -z FEATURE_IDENTIFIER, --feature_identifier FEATURE_IDENTIFIER
                        The feature identifier
  -d DEFINE, --define DEFINE
                        Define the classification problem
  -t TARGET, --target TARGET
                        Define the target domain
  -u UNIQUE, --unique UNIQUE
                        The unique samples to select
  -b, --label_shuffling
                        Label shuffling
  -r RUNS_N, --runs_n RUNS_N
                        The number of runs
  -p RUNS_CV_FOLDS, --runs_cv_folds RUNS_CV_FOLDS
                        The number of cross-validation folds per run
  -w, --set_seed        Setting seed
  -l {rf,svm,lasso,enet}, --learner_type {rf,svm,lasso,enet}
                        The type of learner/classifier
  -i {lasso,enet}, --feature_selection {lasso,enet}
                        The type of feature selection
  -f CV_FOLDS, --cv_folds CV_FOLDS
                        The number of cross-validation folds for model selection
  -g CV_GRID, --cv_grid CV_GRID
                        The parameter grid for model selection
  -s CV_SCORING, --cv_scoring CV_SCORING
                        The scoring function for model selection
  -j FS_GRID, --fs_grid FS_GRID
                        The parameter grid for feature selection
  -e FIGURE_EXTENSION, --figure_extension FIGURE_EXTENSION
                        The extension of output figure
    ```

python classification.py ./abundance_t2d-WT2D.txt results/abundance_t2d-WT2D_rf -d 1:disease:t2d -g \[\] -w

In [2]:
par={'inp_f': './abundance_t2d-WT2D.txt',
     'out_f': 'results/abundance_t2d-WT2D_rf', 
     'feature_identifier': 'k__', 
     'define': '1:disease:t2d', 
     'target': None, 
     'unique': None, 
     'label_shuffling': False, 
     'runs_n': 20, 
     'runs_cv_folds': 10,
     'set_seed': True, 
     'learner_type': None, 
     'feature_selection': None, 
     'cv_folds': None, 
     'cv_grid': '[]',
     'cv_scoring': None, 
     'fs_grid': None, 
     'figure_extension': 'png'}

argsuse = ['classification.py', 
      'data/abundance_t2d-WT2D.txt', 
      'results/abundance_t2d-WT2D_rf', 
      '-d', 
      '1:disease:t2d', 
      '-g', 
      '[]',
      '-w']

In [3]:
# 训练参与的重要参数，见文献method第一和第二部分
class class_params:
    def __init__(self):
        self.learner_type = []
        self.feature_selection = []
        self.cv_folds = []
        self.cv_grid = []
        self.cv_scoring = []
        self.fs_grid = []

def set_class_params(args, l):
    lp = class_params()

    if par['learner_type']:
        lp.learner_type = par['learner_type']
        if (max(l.values.flatten().astype('int')) > 1) & \
                (lp.learner_type != 'svm'):
            lp.learner_type = 'rf'
    else:
        lp.learner_type = 'rf'

    if par['feature_selection']:
        lp.feature_selection = par['feature_selection']
    else:
        lp.feature_selection = 'none'

    if par['cv_folds']:
        lp.cv_folds = int(par['cv_folds'])
    else:
        lp.cv_folds = 5

    if par['cv_grid']:
        lp.cv_grid = eval(par['cv_grid'])
    elif lp.learner_type == 'rf':
        lp.cv_grid = [5, 10, 20, 30, 40, 50, 60,
                      70, 80, 90, 100, 125, 150, 175, 200]
    elif lp.learner_type == 'svm':
        if par['feature_identifier'] == 'k__':
            # lp.cv_grid = [ {'C': [2**s for s in range(-5,16,2)], 'kernel': ['linear']}, {'C': [2**s for s in range(-5,16,2)], 'gamma': [2**s for s in range(3,-15,-2)], 'kernel': ['rbf']} ]
            lp.cv_grid = [
                {'C': [1, 10, 100, 1000],
                 'kernel': ['linear']},
                {'C': [1, 10, 100, 1000],
                 'gamma': [0.001, 0.0001],
                 'kernel': ['rbf']}]
        else:
            lp.cv_grid = [{'C': [1, 10, 100, 1000], 'kernel': ['linear']}]
    elif lp.learner_type == 'lasso':
        lp.cv_grid = [np.logspace(-4, -0.5, 50)]
    elif lp.learner_type == 'enet':
        lp.cv_grid = [np.logspace(-4, -0.5, 50),
                      [0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1.0]]

    if par['fs_grid']:
        lp.fs_grid = eval(par['fs_grid'])
    elif lp.feature_selection == 'lasso':
        lp.fs_grid = [np.logspace(-4, -0.5, 50)]
    elif lp.feature_selection == 'enet':
        lp.fs_grid = [np.logspace(-4, -0.5, 50),
                      [0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1.0]]

    if par['cv_scoring']:
        lp.cv_scoring = par['cv_scoring']
    elif lp.learner_type == 'svm':
        lp.cv_scoring = 'accuracy'

    return lp


In [4]:
f = pd.read_csv(par['inp_f'], sep='\t', header=None, index_col=0) 

In [5]:
f = f.T # 440,608
f.head()

,dataset_name,disease,k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter|s__Methanobrevibacter_smithii,k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter|s__Methanobrevibacter_unclassified,k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanosphaera|s__Methanosphaera_stadtmanae,k__Bacteria|p__Acidobacteria|c__Acidobacteriia|o__Acidobacteriales|f__Acidobacteriaceae|g__Acidobacteriaceae_unclassified,k__Bacteria|p__Actinobacteria|c__Actinobacteria|o__Actinomycetales|f__Actinomycetaceae|g__Actinomyces|s__Actinomyces_graevenitzii,k__Bacteria|p__Actinobacteria|c__Actinobacteria|o__Actinomycetales|f__Actinomycetaceae|g__Actinomyces|s__Actinomyces_odontolyticus,k__Bacteria|p__Actinobacteria|c__Actinobacteria|o__Actinomycetales|f__Actinomycetaceae|g__Actinomyces|s__Actinomyces_turicensis,k__Bacteria|p__Actinobacteria|c__Actinobacteria|o__Actinomycetales|f__Actinomycetaceae|g__Varibaculum|s__Varibaculum_cambriense,...,k__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Xanthomonadales|f__Xanthomonadaceae|g__Xanthomonas|s__Xanthomonas_axonopodis,k__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Xanthomonadales|f__Xanthomonadaceae|g__Xanthomonas|s__Xanthomonas_fuscans,k__Bacteria|p__Tenericutes|c__Mollicutes|o__Mycoplasmatales|f__Mycoplasmataceae|g__Mycoplasma|s__Mycoplasma_bovis,k__Bacteria|p__Bacteroidetes|c__Flavobacteriia|o__Flavobacteriales|f__Flavobacteriaceae|g__Zunongwangia|s__Zunongwangia_profunda,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactobacillales|f__Enterococcaceae|g__Enterococcus|s__Enterococcus_pallens,k__Bacteria|p__Planctomycetes|c__Planctomycetia|o__Planctomycetales|f__Planctomycetaceae|g__Rhodopirellula|s__Rhodopirellula_unclassified,k__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Vibrionales|f__Vibrionaceae|g__Vibrio|s__Vibrio_furnissii,k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o__Bacteroidales|f__Bacteroidaceae|g__Bacteroides|s__Bacteroides_sp_2_2_4,k__Bacteria|p__Firmicutes|c__Bacilli|o__Bacillales|f__Bacillaceae|g__Lysinibacillus|s__Lysinibacillus_fusiformis,k__Bacteria|p__Firmicutes|c__Bacilli|o__Bacillales|f__Bacillaceae|g__Lysinibacillus|s__Lysinibacillus_sphaericus
1,t2dmeta_long,n,0.33364,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,t2dmeta_long,n,0.49776,0.12802,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,t2dmeta_long,n,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,t2dmeta_long,n,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,t2dmeta_long,n,0.49446,0.06786,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
f["disease"]

1        n
2        n
3        n
4        n
5        n
6        n
7        n
8        n
9        n
10       n
11       n
12       n
13       n
14       n
15       n
16       n
17       n
18       n
19       n
20       n
21       n
22       n
23       n
24       n
25       n
26       n
27       n
28       n
29       n
30       n
      ... 
411    t2d
412    t2d
413    t2d
414      n
415    t2d
416    t2d
417    t2d
418    t2d
419    t2d
420    t2d
421    t2d
422      n
423    t2d
424    t2d
425      n
426    t2d
427    t2d
428      n
429    t2d
430    t2d
431      n
432      n
433      n
434      n
435      n
436      n
437      n
438      n
439      n
440      n
Name: disease, Length: 440, dtype: object

In [7]:
d = pd.DataFrame([s.split(':') for s in par['define'].split(',')]) 
# disease列的t2d设置为1 #但由于1是从外部命令获取的，因此变成了字符‘1’
l = pd.DataFrame([0]*len(f))  # 440,1
d.iloc[:,0]=int(d.iloc[:,0])

In [8]:
for i in range(len(d)):
    # f[d.iloc[i, 1]].isin(['t2d']) 将disease列有t2d的，y设置为1
    l[(f[d.iloc[i, 1]].isin(d.iloc[i, 2:])).tolist()] = d.iloc[i, 0]  

In [9]:
# le = prep.LabelEncoder()
# l = pd.DataFrame(le.fit_transform(f.iloc[:, 0]))
# le.transform(f.iloc[:, 0])

In [10]:
runs_n = par['runs_n'] #20
runs_cv_folds = par['runs_cv_folds'] # 10

In [11]:
# 初始化
i_tr = pd.DataFrame(True, index=range(len(f.index)),
                        columns=range(runs_n*runs_cv_folds)) #取440行 200 列  用于训练
i_u = pd.DataFrame(True, index=range(
                len(f.index)), columns=range(runs_n)) # 440,20

In [12]:
# i_tr(shape [440,200])；400存的是样品，200：抽取200次，每一次取样品时，440样品哪些是train，哪些是test
for j in range(runs_n):
    np.random.seed(j)
    ii_u = range(len(f.index)) # f.shape[0] 440
    skf = StratifiedKFold(runs_cv_folds,
                   shuffle=True, random_state=j)
    y_t = l.iloc[i_u.values.T[j], 0]
    x_t = np.zeros(len(y_t))
    for i, (train_index, test_index) in enumerate(skf.split(x_t, y_t)):
                for s in test_index:
                    i_tr[j*runs_cv_folds+i][ii_u[s]] = False

In [13]:
i_tr.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,True,True,True,True,True,False,True,True,True,True,...,True,True,True,False,True,True,True,True,True,True
1,True,True,True,True,True,True,True,False,True,True,...,False,True,True,True,True,True,True,True,True,True
2,True,True,True,True,False,True,True,True,True,True,...,True,True,False,True,True,True,True,True,True,True
3,True,True,True,True,True,False,True,True,True,True,...,True,True,True,True,True,False,True,True,True,True
4,True,True,False,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,False,True


In [14]:
len(i_tr)

440

In [15]:
i_tr = i_tr.values.T #200,440
i_u = i_u.values.T #20,440

In [16]:
# 提取包含 "k__"的fearture  
# f.columns 608
feat = [s for s in f.columns if sum(
    [s2 in s for s2 in par['feature_identifier'].split(':')]) > 0]
    # 含有“unclassified”也保留
if 'unclassified' in f.columns:
        feat.append('unclassified')
# 最后feat 保留606个feature ,提取出来，只去除了2个，同时将feature的值转化为float64
f = f.loc[:, feat].astype('float')
feat[0:2]

['k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter|s__Methanobrevibacter_smithii',
 'k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter|s__Methanobrevibacter_unclassified']

In [17]:
# 特征处理
# 归一化
f = (f-f.min())/(f.max()-f.min())

In [18]:
lp = set_class_params(argsuse, l)
# 训练所采用的参数
lp.__dict__

{'cv_folds': 5,
 'cv_grid': [],
 'cv_scoring': [],
 'feature_selection': 'none',
 'fs_grid': [],
 'learner_type': 'rf'}

In [19]:
f.to_csv("f.csv")
l.to_csv("l.csv")

In [20]:
class feature_importance:
    def __init__(self, feat, p):
        self.feat_sel = feat
        self.imp = np.array([p]*len(feat))


In [21]:
fi = []  #存特征重要因子值的类，跑了两百次的随机森林，存了200个特征因子值的类，#每一个特征的重要性都初始化为1/特征数， 即所有特征的重要性因子累加总和等于1
clf = [] #存了200个模型
p_es = []
l_es = []

# fi.append(feature_importance(feat, 1.0/len(feat)))
# f 440,606 sample, fearture pandas.core.frame.DataFrame

# i_tr 200,440 sample,fearture numpy.ndarray

# i_u 20 440, numpy.ndarray
for j in range(runs_n*runs_cv_folds):
    #每一个特征的重要性都初始化为1/特征数， 即所有特征的重要性因子累加总和等于1
    fi.append(feature_importance(feat, 1.0/len(feat)))
    clf.append(
        RandomForestClassifier(
            n_estimators=500,
            max_depth=None,
            min_samples_split=2,
            n_jobs=-1)
        .fit(
            f.loc[i_tr[j] & i_u[int(j/runs_cv_folds)],
                  fi[j].feat_sel].values,
            l[i_tr[j] & i_u[int(j/runs_cv_folds)]]
            .values.flatten().astype('int')))
    # 将i_tr[j] 是false 的特征矩阵拿来做预测测试，每个list的元素是一个datafrmae
    p_es.append(
        pd.DataFrame(
            clf[j].predict_proba(
                f.loc[~i_tr[j] & i_u[int(j/runs_cv_folds)],
                      fi[j].feat_sel].values)))
    
    l_es.append(pd.DataFrame(
        [list(p_es[j].iloc[i, :]).index(
            max(p_es[j].iloc[i, :])) for i in range(len(p_es[j]))]))

In [22]:
class class_metrics:
    def __init__(self):
        self.accuracy = []
        self.f1 = []
        self.precision = []
        self.recall = []
        self.auc = []
        self.roc_curve = []
        self.confusion_matrix = []

In [23]:
n_clts = len(np.unique(l)) # label类型
cm = class_metrics() #获取200个run的所以评估指标均值

In [24]:
for j in range(runs_n*runs_cv_folds):
    l_ = pd.DataFrame(
        l.loc[l[~i_tr[j] & i_u[int(j / runs_cv_folds)]].index]
    ).values.flatten().astype('int')
    l_es_ = l_es[j].values.flatten().astype('int')
    if (lp.learner_type == 'rf') | (lp.learner_type == 'svm'):
        p_es_pos_ = p_es[j].loc[:, 1].values
    else:
        p_es_pos_ = p_es[j].loc[:, 0].values
    ii_ts_ = [i for i in range(len(i_tr[j])) if i_tr[j][i] == False]

    cm.accuracy.append(metrics.accuracy_score(l_, l_es_))
    cm.f1.append(metrics.f1_score(
        l_, l_es_, pos_label=None, average='weighted'))
    cm.precision.append(metrics.precision_score(
        l_, l_es_, pos_label=None, average='weighted'))
    cm.recall.append(metrics.recall_score(
        l_, l_es_, pos_label=None, average='weighted'))
    if len(np.unique(l_)) == n_clts:
        if n_clts == 2:
            cm.auc.append(metrics.roc_auc_score(l_, p_es_pos_))
            cm.roc_curve.append(metrics.roc_curve(l_, p_es_pos_))
            # print('run/fold\t' + str(int(j/runs_cv_folds)) +
            #                '/' + str(j % runs_cv_folds) + '\n')
            # for i in range(len(cm.roc_curve[-1])):
            #     for i2 in range(len(cm.roc_curve[-1][i])):
            #        print(str(cm.roc_curve[-1][i][i2]) + '\t')
        cm.confusion_matrix.append(metrics.confusion_matrix(
            l_, l_es_, labels=np.unique(l.astype('int'))))

In [25]:
print('#samples\t' + str(int(sum(sum(i_u))/len(i_u))))
print('\n#features\t' + str(len(feat)))
print('\n#runs\t' + str(runs_n))
print('\n#runs_cv_folds\t' + str(runs_cv_folds))

print('\naccuracy\t' + str(np.mean(cm.accuracy)) +
     '\t' + str(np.std(cm.accuracy)))
print('\nf1\t' + str(np.mean(cm.f1)) + '\t' + str(np.std(cm.f1)))
print('\nprecision\t' + str(np.mean(cm.precision)) +
     '\t' + str(np.std(cm.precision)))
print('\nrecall\t' + str(np.mean(cm.recall)) +
     '\t' + str(np.std(cm.recall)))
print('\nauc\t' + str(np.mean(cm.auc)) +
                     '\t' + str(np.std(cm.auc)))

#samples	440

#features	606

#runs	20

#runs_cv_folds	10

accuracy	0.6680832158797275	0.07073727494163164

f1	0.6662960782650884	0.07121877154716355

precision	0.6718650104028089	0.07225949437034287

recall	0.6680832158797275	0.07073727494163164

auc	0.7442876392382322	0.07560311011385139


![](./paper_results.png)

In [25]:
#过滤掉feature_importance小于等于0的
def compute_feature_importance(el, feat, feat_sel, ltype):
    fi = feature_importance(feat, 0.0)
    if ltype == 'rf':
        feature_importances_ = el.feature_importances_
    elif (ltype == 'lasso') | (ltype == 'enet'):
        feature_importances_ = abs(el.coef_)/sum(abs(el.coef_))
    else:
        feature_importances_ = [1.0/len(feat_sel)]*len(feat_sel)
    # ti:feature的index 
    ti = [feat.index(s) for s in feat_sel]
    fi.imp[ti] = feature_importances_
    # 按feature_importances_降序得到index
    t = sorted(range(len(t)), key=lambda s: t[s], reverse=True)
    # 按feature_importances_降序得到fearture 名字
    fi.feat_sel = [feat[ti[s]] for s in t if fi.imp[ti[s]] != 0]

    return fi

In [140]:
fi_f = []
for j in range(runs_n*runs_cv_folds):
    fi_f.append(compute_feature_importance(
        clf[j], feat, fi[j].feat_sel, lp.learner_type))

In [178]:
t=clf[0].feature_importances_
feat_sel=fi[0].feat_sel
sorted(range(len(t)), key=lambda s: t[s], reverse=True)
ti = [feat.index(s) for s in fi[0].feat_sel]
ti[1]

1

In [137]:
pd.DataFrame(
            l.loc[l[~i_tr[0] & i_u[int(0 / runs_cv_folds)]].index]
        ).values.flatten().astype('int')

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1,
       1])

In [45]:
len(l[l[0]==1])

223

In [134]:
pd.DataFrame(
            [l.loc[i] for i in l[~i_tr[0] & i_u[int(0 / runs_cv_folds)]].index]
        ).values.flatten().astype('int')

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1,
       1])